In [1]:
#! jt -r

Reset css and font defaults in:
C:\Users\dhruv\.jupyter\custom &
C:\Users\dhruv\AppData\Roaming\jupyter\nbextensions


# Synthetic Minority Over-sampling TechniquE

In [122]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sea
import math

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score, f1_score


from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor


from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import xgboost as xgb

In [3]:
original_data = pd.read_csv('telco-customer-churn.csv')

In [4]:
original_data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [5]:
original_data_copy = original_data.copy()

In [6]:
colu = original_data.columns.to_list()

In [7]:
conv_colu = colu.copy()
x = ['customerID','SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']
for xx in x:
    conv_colu.remove(xx)
    
conv_colu

['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'Churn']

In [8]:
def int_map(df, column_name):
    dfc = df.copy()
    values = dfc[column_name].unique()
    intmap = {name:val for val,name in enumerate(values)}
    print(intmap)
    dfc[column_name] = dfc[column_name].replace(intmap)
    return(dfc, intmap)

In [9]:
int_map_list = []
for i in conv_colu:
    original_data_copy, int_map_list_temp = int_map(original_data_copy, i)
    int_map_list.append(int_map_list_temp)
original_data_copy

{'Female': 0, 'Male': 1}
{'Yes': 0, 'No': 1}
{'No': 0, 'Yes': 1}
{'No': 0, 'Yes': 1}
{'No phone service': 0, 'No': 1, 'Yes': 2}
{'DSL': 0, 'Fiber optic': 1, 'No': 2}
{'No': 0, 'Yes': 1, 'No internet service': 2}
{'Yes': 0, 'No': 1, 'No internet service': 2}
{'No': 0, 'Yes': 1, 'No internet service': 2}
{'No': 0, 'Yes': 1, 'No internet service': 2}
{'No': 0, 'Yes': 1, 'No internet service': 2}
{'No': 0, 'Yes': 1, 'No internet service': 2}
{'Month-to-month': 0, 'One year': 1, 'Two year': 2}
{'Yes': 0, 'No': 1}
{'Electronic check': 0, 'Mailed check': 1, 'Bank transfer (automatic)': 2, 'Credit card (automatic)': 3}
{'No': 0, 'Yes': 1}


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,29.85,29.85,0
1,5575-GNVDE,1,0,1,0,34,1,1,0,1,...,1,0,0,0,1,1,1,56.95,1889.5,0
2,3668-QPYBK,1,0,1,0,2,1,1,0,1,...,0,0,0,0,0,0,1,53.85,108.15,1
3,7795-CFOCW,1,0,1,0,45,0,0,0,1,...,1,1,0,0,1,1,2,42.30,1840.75,0
4,9237-HQITU,0,0,1,0,2,1,1,1,0,...,0,0,0,0,0,0,0,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,1,0,0,1,24,1,2,0,1,...,1,1,1,1,1,0,1,84.80,1990.5,0
7039,2234-XADUH,0,0,0,1,72,1,2,1,0,...,1,0,1,1,1,0,3,103.20,7362.9,0
7040,4801-JZAZL,0,0,0,1,11,0,0,0,1,...,0,0,0,0,0,0,0,29.60,346.45,0
7041,8361-LTMKD,1,1,0,0,4,1,2,1,0,...,0,0,0,0,0,0,1,74.40,306.6,1


In [10]:
int_map_list

[{'Female': 0, 'Male': 1},
 {'Yes': 0, 'No': 1},
 {'No': 0, 'Yes': 1},
 {'No': 0, 'Yes': 1},
 {'No phone service': 0, 'No': 1, 'Yes': 2},
 {'DSL': 0, 'Fiber optic': 1, 'No': 2},
 {'No': 0, 'Yes': 1, 'No internet service': 2},
 {'Yes': 0, 'No': 1, 'No internet service': 2},
 {'No': 0, 'Yes': 1, 'No internet service': 2},
 {'No': 0, 'Yes': 1, 'No internet service': 2},
 {'No': 0, 'Yes': 1, 'No internet service': 2},
 {'No': 0, 'Yes': 1, 'No internet service': 2},
 {'Month-to-month': 0, 'One year': 1, 'Two year': 2},
 {'Yes': 0, 'No': 1},
 {'Electronic check': 0,
  'Mailed check': 1,
  'Bank transfer (automatic)': 2,
  'Credit card (automatic)': 3},
 {'No': 0, 'Yes': 1}]

In [11]:
all_nulls = []
for x in range(len(original_data_copy)):
    if original_data_copy['TotalCharges'][x] == ' ' or None:
        print(x)
        all_nulls.append(x)
        
original_data_copy.drop(all_nulls, inplace=True)
original_data_copy

488
753
936
1082
1340
3331
3826
4380
5218
6670
6754


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,29.85,29.85,0
1,5575-GNVDE,1,0,1,0,34,1,1,0,1,...,1,0,0,0,1,1,1,56.95,1889.5,0
2,3668-QPYBK,1,0,1,0,2,1,1,0,1,...,0,0,0,0,0,0,1,53.85,108.15,1
3,7795-CFOCW,1,0,1,0,45,0,0,0,1,...,1,1,0,0,1,1,2,42.30,1840.75,0
4,9237-HQITU,0,0,1,0,2,1,1,1,0,...,0,0,0,0,0,0,0,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,1,0,0,1,24,1,2,0,1,...,1,1,1,1,1,0,1,84.80,1990.5,0
7039,2234-XADUH,0,0,0,1,72,1,2,1,0,...,1,0,1,1,1,0,3,103.20,7362.9,0
7040,4801-JZAZL,0,0,0,1,11,0,0,0,1,...,0,0,0,0,0,0,0,29.60,346.45,0
7041,8361-LTMKD,1,1,0,0,4,1,2,1,0,...,0,0,0,0,0,0,1,74.40,306.6,1


In [12]:
original_data_copy['TotalCharges'] = original_data_copy['TotalCharges'].astype('float')

In [13]:
del_colu = ['customerID', 'Churn', 'MonthlyCharges', 'TotalCharges']
graph_colu = colu.copy()
for x in del_colu:
    graph_colu.remove(x)
    
graph_colu

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod']

In [14]:
'''data_copy_churn_0 = original_data_copy[original_data_copy['Churn']==0]
data_copy_churn_1 = original_data_copy[original_data_copy['Churn']==1]'''

"data_copy_churn_0 = original_data_copy[original_data_copy['Churn']==0]\ndata_copy_churn_1 = original_data_copy[original_data_copy['Churn']==1]"

In [15]:
def count_plot(x):
    #original_data_copy[x].value_counts().plot.bar()
    print(original_data_copy[x].value_counts(),'\n')

In [16]:
for x in graph_colu:
    print(x)
    count_plot(x)
    #print('---------------------------------------------------------------------\n')

gender
1    3549
0    3483
Name: gender, dtype: int64 

SeniorCitizen
0    5890
1    1142
Name: SeniorCitizen, dtype: int64 

Partner
1    3639
0    3393
Name: Partner, dtype: int64 

Dependents
0    4933
1    2099
Name: Dependents, dtype: int64 

tenure
1     613
72    362
2     238
3     200
4     176
     ... 
38     59
28     57
39     56
44     51
36     50
Name: tenure, Length: 72, dtype: int64 

PhoneService
1    6352
0     680
Name: PhoneService, dtype: int64 

MultipleLines
1    3385
2    2967
0     680
Name: MultipleLines, dtype: int64 

InternetService
1    3096
0    2416
2    1520
Name: InternetService, dtype: int64 

OnlineSecurity
0    3497
1    2015
2    1520
Name: OnlineSecurity, dtype: int64 

OnlineBackup
1    3087
0    2425
2    1520
Name: OnlineBackup, dtype: int64 

DeviceProtection
0    3094
1    2418
2    1520
Name: DeviceProtection, dtype: int64 

TechSupport
0    3472
1    2040
2    1520
Name: TechSupport, dtype: int64 

StreamingTV
0    2809
1    2703
2    152

###### Features with unbalanced values

SeniorCitizen

Dependents

PhoneService

MultipleLines

InternetService

OnlineBackup

DeviceProtection

TechSupport

Contract

PaperlessBilling



# Training model on the original data

In [17]:
xtrain, xtest, ytrain, ytest = train_test_split(original_data_copy[colu[1:len(colu)-1]], original_data_copy['Churn'], test_size=0.25)

In [18]:
xtrain

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
3095,1,0,1,0,11,1,1,1,0,1,1,0,0,1,0,1,0,84.80,888.75
6767,1,0,0,0,57,1,1,0,1,1,0,1,0,0,0,1,0,55.70,3171.60
3121,0,1,1,0,16,1,1,0,0,1,0,1,1,0,0,1,0,59.40,1023.90
393,0,0,1,0,19,1,1,2,2,2,2,2,2,2,0,1,1,19.90,367.55
3019,0,0,0,1,22,1,1,0,0,0,0,1,0,0,0,1,0,55.15,1206.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,25.05,25.05
4249,0,0,0,1,10,1,1,0,0,0,1,0,0,1,1,0,1,65.90,660.05
1852,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,1,3,46.20,46.20
4863,1,0,0,1,57,1,1,2,2,2,2,2,2,2,2,1,1,20.75,1123.15


In [19]:
print(len(xtrain),'\n----------\n')
for x in graph_colu:
    print(x)
    print(xtrain[x].value_counts(),'\n')

5274 
----------

gender
1    2660
0    2614
Name: gender, dtype: int64 

SeniorCitizen
0    4411
1     863
Name: SeniorCitizen, dtype: int64 

Partner
1    2692
0    2582
Name: Partner, dtype: int64 

Dependents
0    3691
1    1583
Name: Dependents, dtype: int64 

tenure
1     463
72    282
2     175
3     156
4     134
     ... 
59     43
28     43
36     39
39     39
44     34
Name: tenure, Length: 72, dtype: int64 

PhoneService
1    4775
0     499
Name: PhoneService, dtype: int64 

MultipleLines
1    2533
2    2242
0     499
Name: MultipleLines, dtype: int64 

InternetService
1    2332
0    1815
2    1127
Name: InternetService, dtype: int64 

OnlineSecurity
0    2634
1    1513
2    1127
Name: OnlineSecurity, dtype: int64 

OnlineBackup
1    2325
0    1822
2    1127
Name: OnlineBackup, dtype: int64 

DeviceProtection
0    2332
1    1815
2    1127
Name: DeviceProtection, dtype: int64 

TechSupport
0    2606
1    1541
2    1127
Name: TechSupport, dtype: int64 

StreamingTV
0    2136


In [20]:
ytrain

3095    0
6767    0
3121    1
393     0
3019    1
       ..
3185    1
4249    0
1852    1
4863    0
1429    0
Name: Churn, Length: 5274, dtype: int64

#### Naive bayes

In [21]:
naive = GaussianNB()
model = naive.fit(xtrain, ytrain)

In [22]:
#naive_bayes_original_data_pred = naive.predict(xtest.iloc[0].to_numpy().reshape(1,-1))
naive_bayes_original_data_pred = naive.predict(xtest)

In [23]:
naive_bayes_original_data_pred

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [24]:
naive_bayes_original_data_accuracy = accuracy_score(ytest,naive_bayes_original_data_pred)
naive_bayes_original_data_accuracy

0.7155858930602957

In [25]:
naive_bayes_original_data_confusion_matrix = confusion_matrix(ytest, naive_bayes_original_data_pred)
naive_bayes_original_data_confusion_matrix.astype('float')/ naive_bayes_original_data_confusion_matrix.sum(axis=1)[:, np.newaxis]

array([[0.68199532, 0.31800468],
       [0.19368421, 0.80631579]])

In [26]:
naive_bayes_original_data_precision = precision_score(ytest, naive_bayes_original_data_pred)
print(naive_bayes_original_data_precision)

0.48419721871049304


In [27]:
naive_bayes_original_data_recall = recall_score(ytest, naive_bayes_original_data_pred)
print(naive_bayes_original_data_recall)

0.8063157894736842


In [28]:
naive_bayes_original_data_f1 = f1_score(ytest, naive_bayes_original_data_pred)
print(naive_bayes_original_data_f1)

0.6050552922590837


#### Logistic Regression

In [29]:
log = LogisticRegression(max_iter=1000000)
log.fit(xtrain,ytrain)

LogisticRegression(max_iter=1000000)

In [30]:
log_reg_original_data_pred = log.predict(xtest)

In [31]:
log_reg_original_data_pred

array([1, 0, 0, ..., 0, 0, 1], dtype=int64)

In [32]:
log_reg_original_data_accuracy = accuracy_score(ytest, log_reg_original_data_pred)
log_reg_original_data_accuracy

0.8128555176336746

In [33]:
log_reg_original_data_confusion_matrix = confusion_matrix(ytest, log_reg_original_data_pred)
log_reg_original_data_confusion_matrix.astype('float')/ log_reg_original_data_confusion_matrix.sum(axis=1)[:, np.newaxis]

array([[0.90335152, 0.09664848],
       [0.43157895, 0.56842105]])

In [34]:
log_reg_original_data_precision = precision_score(ytest, log_reg_original_data_pred)
print(log_reg_original_data_precision)

0.6852791878172588


In [35]:
log_reg_original_data_recall = recall_score(ytest, log_reg_original_data_pred)
print(log_reg_original_data_recall)

0.5684210526315789


In [36]:
log_reg_original_data_f1 = f1_score(ytest, log_reg_original_data_pred)
print(log_reg_original_data_f1)

0.621403912543153


#### Random Forests

In [37]:
ran_for = RandomForestClassifier()
ran_for.fit(xtrain, ytrain)

RandomForestClassifier()

In [38]:
ran_for_original_data_pred = ran_for.predict(xtest)
ran_for_original_data_pred

array([1, 1, 0, ..., 0, 1, 0], dtype=int64)

In [39]:
ran_for_original_data_accuracy = accuracy_score(ytest, ran_for_original_data_pred)
ran_for_original_data_accuracy

0.7889647326507395

In [40]:
ran_for_original_data_confusion_matrix = confusion_matrix(ytest, ran_for_original_data_pred)
ran_for_original_data_confusion_matrix.astype('float')/ ran_for_original_data_confusion_matrix.sum(axis=1)[:, np.newaxis]

array([[0.90101325, 0.09898675],
       [0.51368421, 0.48631579]])

In [41]:
ran_for_original_data_precision = precision_score(ytest, ran_for_original_data_pred)
print(ran_for_original_data_precision)

0.6452513966480447


In [42]:
ran_for_original_data_recall = recall_score(ytest, ran_for_original_data_pred)
print(ran_for_original_data_recall)

0.4863157894736842


In [43]:
ran_for_original_data_f1 = f1_score(ytest, ran_for_original_data_pred)
print(ran_for_original_data_f1)

0.5546218487394957


In [143]:
n_estimators = [5,20,50,100, 150, 200, 250, 300] # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10, 12, 14] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4, 5, 6] # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points

random_grid = {'n_estimators': n_estimators,
'max_features': max_features,
'max_depth': max_depth,
'min_samples_split': min_samples_split,
'min_samples_leaf': min_samples_leaf,
'bootstrap': bootstrap}

In [144]:
rf_random = RandomizedSearchCV(estimator = ran_for,param_distributions = random_grid,n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)

In [145]:
rf_random.fit(xtrain, ytrain)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      120],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 3, 4, 5, 6],
                                        'min_samples_split': [2, 6, 10, 12, 14],
                                        'n_estimators': [5, 20, 50, 100, 150,
                                                         200, 250, 300]},
                   random_state=35, verbose=2)

In [147]:
rf_random.best_params_

{'n_estimators': 300,
 'min_samples_split': 14,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 70,
 'bootstrap': True}

In [148]:
ran_for_tuned = RandomForestClassifier(n_estimators = 300, min_samples_split=14, min_samples_leaf=4, 
                                       max_features='sqrt', max_depth=70, bootstrap=True)
ran_for_tuned.fit(xtrain_smot, ytrain_smot)

ran_for_tuned_original_data_pred = ran_for.predict(xtest_smot)

ran_for_tuned_original_data_accuracy = accuracy_score(ytest_smot, ran_for_tuned_original_data_pred)
print('accuracy: ', ran_for_tuned_original_data_accuracy)

ran_for_tuned_original_data_precision = precision_score(ytest_smot, ran_for_tuned_original_data_pred)
print('precision: ', ran_for_tuned_original_data_precision)

ran_for_tuned_original_data_recall = recall_score(ytest_smot, ran_for_tuned_original_data_pred)
print('recall: ', ran_for_tuned_smot_recall)

ran_for_tuned_original_data_f1 = f1_score(ytest_smot, ran_for_tuned_original_data_pred)
print('f1: ', ran_for_tuned_original_data_f1)


accuracy:  0.920135527589545
precision:  0.9565217391304348
recall:  0.8791423001949318
f1:  0.9162011173184358


#### XGBoost

In [44]:
xg_boost = XGBRegressor()
xg_boost.fit(xtrain, ytrain)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [45]:
xgb_original_data_pred = xg_boost.predict(xtest)
xgb_original_data_pred = np.round(xgb_original_data_pred)

In [48]:
xgb_original_data_accuracy = accuracy_score(ytest, xgb_original_data_pred)
xgb_original_data_accuracy

0.7821387940841866

In [55]:
xgb_original_data_confusion_matrix = confusion_matrix(ytest, xgb_original_data_pred)
xgb_original_data_confusion_matrix.astype('float')/ xgb_original_data_confusion_matrix.sum(axis=1)[:,np.newaxis]

array([[0.87685113, 0.12314887],
       [0.47368421, 0.52631579]])

In [56]:
xgb_original_data_precision = precision_score(ytest, xgb_original_data_pred)
print(xgb_original_data_precision)

0.6127450980392157


In [57]:
xgb_original_data_recall = recall_score(ytest, xgb_original_data_pred)
print(xgb_original_data_recall)

0.5263157894736842


In [58]:
xgb_original_data_f1 = f1_score(ytest, xgb_original_data_pred)
print(xgb_original_data_f1)

0.5662514156285392


### Brief comparision (original data)

    model                Accuracy       Precision       Recall       F1 score

    Naive bayes          0.7224         0.507           0.802        0.621

    Logistic regression  0.816          0.259           0.194        0.221

    Random forest        0.795          0.25            0.172        0.203

    XGBoost              0.7633         0.611           0.462        0.526

Accuracy: quality of prediction

Prediction: quality of positive predictions

recall: ratio of actually positive values to precicted positive values

F1: harmonic mean of precision and recall 

---

# SMOTE

In [59]:
smot = SMOTE(random_state=101)
x_resamples, y_resamples = smot.fit_resample(original_data_copy[colu[1:len(colu)-1]], original_data_copy['Churn'])

In [60]:
x_resamples

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,29.850000,29.850000
1,1,0,1,0,34,1,1,0,1,1,1,0,0,0,1,1,1,56.950000,1889.500000
2,1,0,1,0,2,1,1,0,1,0,0,0,0,0,0,0,1,53.850000,108.150000
3,1,0,1,0,45,0,0,0,1,1,1,1,0,0,1,1,2,42.300000,1840.750000
4,0,0,1,0,2,1,1,1,0,1,0,0,0,0,0,0,0,70.700000,151.650000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10321,0,0,1,0,16,1,1,0,0,1,1,0,0,0,0,0,0,58.209558,937.864485
10322,1,0,0,0,54,1,1,1,1,0,0,0,0,1,1,0,2,109.528404,6109.574554
10323,0,0,1,0,1,1,1,1,0,1,0,0,0,0,0,0,0,70.114711,70.114711
10324,1,0,0,0,7,1,1,1,0,0,0,0,1,1,0,0,0,97.604865,666.618046


In [61]:
y_resamples

0        0
1        0
2        1
3        0
4        1
        ..
10321    1
10322    1
10323    1
10324    1
10325    1
Name: Churn, Length: 10326, dtype: int64

In [62]:
print(len(x_resamples),'\n----------\n')
for x in graph_colu:
    print(x)
    print(x_resamples[x].value_counts(),'\n')

10326 
----------

gender
0    5852
1    4474
Name: gender, dtype: int64 

SeniorCitizen
0    8923
1    1403
Name: SeniorCitizen, dtype: int64 

Partner
0    5195
1    5131
Name: Partner, dtype: int64 

Dependents
0    8106
1    2220
Name: Dependents, dtype: int64 

tenure
1     1270
2      469
3      385
72     362
4      322
      ... 
38      75
59      73
39      73
44      72
36      70
Name: tenure, Length: 72, dtype: int64 

PhoneService
1    9253
0    1073
Name: PhoneService, dtype: int64 

MultipleLines
1    5334
2    3952
0    1040
Name: MultipleLines, dtype: int64 

InternetService
1    5294
0    3350
2    1682
Name: InternetService, dtype: int64 

OnlineSecurity
0    6485
1    2159
2    1682
Name: OnlineSecurity, dtype: int64 

OnlineBackup
1    4827
0    3817
2    1682
Name: OnlineBackup, dtype: int64 

DeviceProtection
0    5751
1    2893
2    1682
Name: DeviceProtection, dtype: int64 

TechSupport
0    6456
1    2188
2    1682
Name: TechSupport, dtype: int64 

StreamingT

#### SMOTE-NC

In [63]:
xtrain.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges'],
      dtype='object')

In [64]:
smotnc = SMOTENC([0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17], random_state=101)
x_resamples_nc, y_resamples_nc = smotnc.fit_resample(original_data_copy[colu[1:len(colu)-1]], original_data_copy['Churn'])

In [65]:
x_resamples_nc

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,29.85,29.850000
1,1,0,1,0,34,1,1,0,1,1,1,0,0,0,1,1,1,56.95,1889.500000
2,1,0,1,0,2,1,1,0,1,0,0,0,0,0,0,0,1,53.85,108.150000
3,1,0,1,0,45,0,0,0,1,1,1,1,0,0,1,1,2,42.30,1840.750000
4,0,0,1,0,2,1,1,1,0,1,0,0,0,0,0,0,0,70.70,151.650000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10321,0,0,1,0,17,1,2,0,0,1,0,0,0,0,0,0,1,55.70,954.080850
10322,0,0,0,0,53,1,1,1,1,0,0,0,1,1,1,0,3,105.40,5773.064110
10323,0,0,1,0,1,1,1,1,0,1,0,0,0,0,0,0,0,70.10,70.114711
10324,1,0,1,0,6,1,1,1,0,0,0,0,1,1,0,0,0,96.25,593.414894


In [66]:
print(len(x_resamples_nc),'\n----------\n')
for x in graph_colu:
    print(x)
    print(x_resamples_nc[x].value_counts(),'\n')

10326 
----------

gender
1    5243
0    5083
Name: gender, dtype: int64 

SeniorCitizen
0    8479
1    1847
Name: SeniorCitizen, dtype: int64 

Partner
1    5853
0    4473
Name: Partner, dtype: int64 

Dependents
0    7829
1    2497
Name: Dependents, dtype: int64 

tenure
1     1334
2      479
3      370
72     364
4      317
      ... 
36      73
45      73
38      72
59      70
44      62
Name: tenure, Length: 72, dtype: int64 

PhoneService
1    9342
0     984
Name: PhoneService, dtype: int64 

MultipleLines
1    4883
2    4459
0     984
Name: MultipleLines, dtype: int64 

InternetService
1    5451
0    3144
2    1731
Name: InternetService, dtype: int64 

OnlineSecurity
0    6280
1    2315
2    1731
Name: OnlineSecurity, dtype: int64 

OnlineBackup
1    5330
0    3265
2    1731
Name: OnlineBackup, dtype: int64 

DeviceProtection
0    5274
1    3321
2    1731
Name: DeviceProtection, dtype: int64 

TechSupport
0    6207
1    2388
2    1731
Name: TechSupport, dtype: int64 

StreamingT

---

## Using SMOT data

In [67]:
xtrain_smot, xtest_smot, ytrain_smot, ytest_smot = train_test_split(x_resamples, y_resamples, test_size=0.20)

#### naive bayes smot

In [68]:
naive = GaussianNB()
model = naive.fit(xtrain_smot, ytrain_smot)
#naive_bayes_original_data_pred = naive.predict(xtest.iloc[0].to_numpy().reshape(1,-1))
naive_bayes_smot_pred = naive.predict(xtest_smot)

naive_bayes_smot_accuracy = accuracy_score(ytest_smot,naive_bayes_smot_pred)
print('accuracy: ', naive_bayes_smot_accuracy)

naive_bayes_smot_precision = precision_score(ytest_smot, naive_bayes_smot_pred)
print('precision: ', naive_bayes_smot_precision)

naive_bayes_smot_recall = recall_score(ytest_smot, naive_bayes_smot_pred)
print('recall: ', naive_bayes_smot_recall)

naive_bayes_smot_f1 = f1_score(ytest_smot, naive_bayes_smot_pred)
print('f1: ', naive_bayes_smot_f1)

naive_bayes_smot_confusion_matrix = confusion_matrix(ytest_smot, naive_bayes_smot_pred)
naive_bayes_smot_confusion_matrix.astype('float')/ naive_bayes_smot_confusion_matrix.sum(axis=1)[:, np.newaxis]

accuracy:  0.7894482090997096
precision:  0.7432098765432099
recall:  0.8801169590643275
f1:  0.8058902275769746


array([[0.7       , 0.3       ],
       [0.11988304, 0.88011696]])

#### Logistic regression smot

In [69]:
log = LogisticRegression(max_iter=1000000)
log.fit(xtrain_smot, ytrain_smot)
log_reg_smot_pred = log.predict(xtest_smot)

log_reg_smot_accuracy = accuracy_score(ytest_smot, log_reg_smot_pred)
print('accuracy: ', log_reg_smot_accuracy)

log_reg_smot_precision = precision_score(ytest_smot, log_reg_smot_pred)
print('precision: ', log_reg_smot_precision)

log_reg_smot_recall = recall_score(ytest_smot, log_reg_smot_pred)
print('recall: ', log_reg_smot_recall)

log_reg_smot_f1 = f1_score(ytest_smot, log_reg_smot_pred)
print('f1: ', log_reg_smot_f1)

log_reg_smot_confusion_matrix = confusion_matrix(ytest_smot, log_reg_smot_pred)
log_reg_smot_confusion_matrix.astype('float')/ log_reg_smot_confusion_matrix.sum(axis=1)[:, np.newaxis]

accuracy:  0.8112294288480155
precision:  0.797752808988764
recall:  0.8304093567251462
f1:  0.8137535816618909


array([[0.79230769, 0.20769231],
       [0.16959064, 0.83040936]])

#### Random forest smot

In [70]:
ran_for = RandomForestClassifier()
ran_for.fit(xtrain_smot, ytrain_smot)

ran_for_smot_pred = ran_for.predict(xtest_smot)

ran_for_smot_accuracy = accuracy_score(ytest_smot, ran_for_smot_pred)
print('accuracy: ', ran_for_smot_accuracy)

ran_for_smot_precision = precision_score(ytest_smot, ran_for_smot_pred)
print('precision: ', ran_for_smot_precision)

ran_for_smot_recall = recall_score(ytest_smot, ran_for_smot_pred)
print('recall: ', ran_for_smot_recall)

ran_for_smot_f1 = f1_score(ytest_smot, ran_for_smot_pred)
print('f1: ', ran_for_smot_f1)


ran_for_smot_confusion_matrix = confusion_matrix(ytest_smot, ran_for_smot_pred)
ran_for_smot_confusion_matrix.astype('float')/ ran_for_smot_confusion_matrix.sum(axis=1)[:, np.newaxis]

accuracy:  0.8441432720232332
precision:  0.8333333333333334
recall:  0.8576998050682261
f1:  0.845341018251681


array([[0.83076923, 0.16923077],
       [0.14230019, 0.85769981]])

### Hyperparameter tuning for Random forest

In [110]:
n_estimators = [5,20,50,100, 150, 200, 250, 300] # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10, 12, 14] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4, 5, 6] # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points

random_grid = {'n_estimators': n_estimators,
'max_features': max_features,
'max_depth': max_depth,
'min_samples_split': min_samples_split,
'min_samples_leaf': min_samples_leaf,
'bootstrap': bootstrap}

In [111]:
rf_random = RandomizedSearchCV(estimator = ran_for,param_distributions = random_grid,n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)

In [112]:
rf_random.fit(xtrain_smot, ytrain_smot)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      120],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 3, 4, 5, 6],
                                        'min_samples_split': [2, 6, 10, 12, 14],
                                        'n_estimators': [5, 20, 50, 100, 150,
                                                         200, 250, 300]},
                   random_state=35, verbose=2)

In [113]:
rf_random.best_params_

{'n_estimators': 250,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 120,
 'bootstrap': True}

In [114]:
ran_for_tuned = RandomForestClassifier(n_estimators = 250, min_samples_split=2, min_samples_leaf=1, 
                                       max_features='sqrt', max_depth=120, bootstrap=True)
ran_for_tuned.fit(xtrain_smot, ytrain_smot)

ran_for_tuned_smot_pred = ran_for.predict(xtest_smot)

ran_for_tuned_smot_accuracy = accuracy_score(ytest_smot, ran_for_smot_pred)
print('accuracy: ', ran_for_tuned_smot_accuracy)

ran_for_tuned_smot_precision = precision_score(ytest_smot, ran_for_tuned_smot_pred)
print('precision: ', ran_for_tuned_smot_precision)

ran_for_tuned_smot_recall = recall_score(ytest_smot, ran_for_tuned_smot_pred)
print('recall: ', ran_for_tuned_smot_recall)

ran_for_tuned_smot_f1 = f1_score(ytest_smot, ran_for_tuned_smot_pred)
print('f1: ', ran_for_tuned_smot_f1)


accuracy:  0.8441432720232332
precision:  0.9565217391304348
recall:  0.8791423001949318
f1:  0.9162011173184358


#### xgboost smot

In [71]:
xg_boost = XGBRegressor()
xg_boost.fit(xtrain_smot, ytrain_smot)

xgb_smot_prediction = xg_boost.predict(xtest_smot)
xgb_smot_pred = np.round(xgb_smot_prediction)

xgb_smot_accuracy = accuracy_score(ytest_smot, xgb_smot_pred)
print('accuracy: ', xgb_smot_accuracy)

xgb_smot_precision = precision_score(ytest_smot, xgb_smot_pred)
print('precision: ', xgb_smot_precision)

xgb_smot_recall = recall_score(ytest_smot, xgb_smot_pred)
print('recall: ', xgb_smot_recall)

xgb_smot_f1 = f1_score(ytest_smot, xgb_smot_pred)
print('f1: ', xgb_smot_f1)

xgb_smot_confusion_matrix = confusion_matrix(ytest_smot, xgb_smot_pred)
xgb_smot_confusion_matrix.astype('float')/ xgb_smot_confusion_matrix.sum(axis=1)[:,np.newaxis]

accuracy:  0.8397870280735721
precision:  0.8377065111758989
recall:  0.8401559454191033
f1:  0.8389294403892944


array([[0.83942308, 0.16057692],
       [0.15984405, 0.84015595]])

### Brief comparision (original data)

    model                Accuracy       Precision       Recall       F1 score

    Naive bayes          0.7224         0.507           0.802        0.621

    Logistic regression  0.816          0.259           0.194        0.221

    Random forest        0.795          0.25            0.172        0.203

    XGBoost              0.7633         0.611           0.462        0.526

### Hyperparameter tuning for XGBoost

### Brief comparision (SMOT data)

    model                Accuracy       Precision       Recall       F1 score

    Naive bayes          0.733          0.732           0.854        0.788       

    Logistic regression  0.862          0.259           0.863        0.831

    Random forest        0.849          0.25            0.847        0.847

    XGBoost              0.846          0.838           0.853        0.845

In [126]:
def objective(space):
    clf=xgb.XGBClassifier(n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), 
                          gamma = space['gamma'],reg_alpha = int(space['reg_alpha']),
                          min_child_weight=int(space['min_child_weight']),
                          colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [(xtrain_smot, ytrain_smot), (xtest_smot, ytest_smot)]
    
    clf.fit(xtrain_smot, ytrain_smot,eval_set=evaluation, eval_metric="auc",early_stopping_rounds=10,verbose=False)
    

    xg_triasl_pred = clf.predict(xtest_smot)
    accuracy = accuracy_score(ytest_smot, xg_triasl_pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [136]:
xgt = XGBRegressor(best_hyperparams)
#xgt.set_params(**best_hyperparams)

model = xgt
xgt.fit(xtrain_smot, ytrain_smot)

xgb_tuned_smot_prediction = xgt.predict(xtest_smot)
xgb_tuned_smot_pred = np.round(xgb_tuned_smot_prediction)

xgb_tuned_smot_accuracy = accuracy_score(ytest_smot, xgb_tuned_smot_pred)
print('accuracy: ', xgb_tuned_smot_accuracy)

xgb_tuned_smot_precision = precision_score(ytest_smot, xgb_tuned_smot_pred)
print('precision: ', xgb_tuned_smot_precision)

xgb_tuned_smot_recall = recall_score(ytest_smot, xgb_tuned_smot_pred)
print('recall: ', xgb_tuned_smot_recall)

xgb_tuned_smot_f1 = f1_score(ytest_smot, xgb_tuned_smot_pred)
print('f1: ', xgb_tuned_smot_f1)

xgb_tuned_smot_confusion_matrix = confusion_matrix(ytest_smot, xgb_tuned_smot_pred)
xgb_tuned_smot_confusion_matrix.astype('float')/ xgb_tuned_smot_confusion_matrix.sum(axis=1)[:,np.newaxis]

D:\Folder\main\Anaconda\lib\site-packages\xgboost\core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning


XGBoostError: [21:04:22] C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\objective\objective.cc:26: Unknown objective function: `{'colsample_bytree': 0.9905250031928715, 'gamma': 1.0311980465001422, 'max_depth': 5.0, 'min_child_weight': 9.0, 'reg_alpha': 41.0, 'reg_lambda': 0.6118439059582054}`
Objective candidate: survival:aft
Objective candidate: binary:hinge
Objective candidate: multi:softmax
Objective candidate: multi:softprob
Objective candidate: rank:pairwise
Objective candidate: rank:ndcg
Objective candidate: rank:map
Objective candidate: reg:squarederror
Objective candidate: reg:squaredlogerror
Objective candidate: reg:logistic
Objective candidate: reg:pseudohubererror
Objective candidate: binary:logistic
Objective candidate: binary:logitraw
Objective candidate: reg:linear
Objective candidate: count:poisson
Objective candidate: survival:cox
Objective candidate: reg:gamma
Objective candidate: reg:tweedie


XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.9905250031928715,
             enable_categorical=False, gamma=1.0311980465001422, gpu_id=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_delta_step=None, max_depth=5.0,
             min_child_weight=9.0, missing=nan, monotone_constraints=None,
             n_e...el_tree=None,
             objective={'colsample_bytree': 0.9905250031928715,
                        'gamma': 1.0311980465001422, 'max_depth': 5.0,
                        'min_child_weight': 9.0, 'reg_alpha': 41.0,
                        'reg_lambda': 0.6118439059582054},
             predictor=None, random_state=None, reg_alpha=41.0,
             reg_lambda=0.6118439059582054, scale_pos_weight=None,
             subsample=None, tree_method=None, validate_parameters=None,
             verbosity=None)

---

In [118]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

## Using SMOTE-NC data

In [127]:
tr = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = tr)

  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

D:\Folder\main\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



SCORE:                                                                                                                 
0.8000968054211036                                                                                                     
  1%|▍                                              | 1/100 [00:00<01:22,  1.20trial/s, best loss: -0.8000968054211036]

D:\Folder\main\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



SCORE:                                                                                                                 
0.7996127783155856                                                                                                     
  2%|▉                                              | 2/100 [00:01<01:21,  1.21trial/s, best loss: -0.8000968054211036]

...

SCORE:                                                                                                                 
0.8039690222652468                                                                                                     
 98%|█████████████████████████████████████████████ | 98/100 [00:24<00:00,  4.22trial/s, best loss: -0.8068731848983544]

D:\Folder\main\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



SCORE:                                                                                                                 
0.8005808325266215                                                                                                     
 99%|█████████████████████████████████████████████▌| 99/100 [00:25<00:00,  4.13trial/s, best loss: -0.8068731848983544]

D:\Folder\main\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



SCORE:                                                                                                                 
0.8025169409486931                                                                                                     
100%|█████████████████████████████████████████████| 100/100 [00:25<00:00,  3.96trial/s, best loss: -0.8068731848983544]


In [72]:
xtrain_smotnc, xtest_smotnc, ytrain_smotnc, ytest_smotnc = train_test_split(x_resamples_nc, y_resamples_nc, test_size=0.20)

In [128]:
best_hyperparams

{'colsample_bytree': 0.9905250031928715,
 'gamma': 1.0311980465001422,
 'max_depth': 5.0,
 'min_child_weight': 9.0,
 'reg_alpha': 41.0,
 'reg_lambda': 0.6118439059582054}

#### naive bayes smotnc

In [73]:
naive = GaussianNB()
model = naive.fit(xtrain_smotnc, ytrain_smotnc)
#naive_bayes_original_data_pred = naive.predict(xtest.iloc[0].to_numpy().reshape(1,-1))
naive_bayes_smotnc_pred = naive.predict(xtest_smotnc)

naive_bayes_smotnc_accuracy = accuracy_score(ytest_smotnc,naive_bayes_smotnc_pred)
print('accuracy: ', naive_bayes_smotnc_accuracy)

naive_bayes_smotnc_precision = precision_score(ytest_smotnc, naive_bayes_smotnc_pred)
print('precision: ', naive_bayes_smot_precision)

naive_bayes_smotnc_recall = recall_score(ytest_smotnc, naive_bayes_smotnc_pred)
print('recall: ', naive_bayes_smotnc_recall)

naive_bayes_smotnc_f1 = f1_score(ytest_smotnc, naive_bayes_smotnc_pred)
print('f1: ', naive_bayes_smotnc_f1)

naive_bayes_smotnc_confusion_matrix = confusion_matrix(ytest_smotnc, naive_bayes_smotnc_pred)
naive_bayes_smotnc_confusion_matrix.astype('float')/ naive_bayes_smotnc_confusion_matrix.sum(axis=1)[:, np.newaxis]

accuracy:  0.7429816069699903
precision:  0.7432098765432099
recall:  0.8475551294343241
f1:  0.7690300130491518


array([[0.63636364, 0.36363636],
       [0.15244487, 0.84755513]])

#### Logistic regression smotnc

In [74]:
log = LogisticRegression(max_iter=1000000)
log.fit(xtrain_smotnc, ytrain_smotnc)
log_reg_smotnc_pred = log.predict(xtest_smotnc)

log_reg_smotnc_accuracy = accuracy_score(ytest_smotnc, log_reg_smotnc_pred)
print('accuracy: ', log_reg_smotnc_accuracy)

log_reg_smotnc_precision = precision_score(ytest_smotnc, log_reg_smotnc_pred)
print('precision: ', log_reg_smot_precision)

log_reg_smotnc_recall = recall_score(ytest_smotnc, log_reg_smotnc_pred)
print('recall: ', log_reg_smotnc_recall)

log_reg_smotnc_f1 = f1_score(ytest_smotnc, log_reg_smotnc_pred)
print('f1: ', log_reg_smotnc_f1)

log_reg_smotnc_confusion_matrix = confusion_matrix(ytest_smotnc, log_reg_smotnc_pred)
log_reg_smotnc_confusion_matrix.astype('float')/ log_reg_smotnc_confusion_matrix.sum(axis=1)[:, np.newaxis]

accuracy:  0.7831558567279767
precision:  0.797752808988764
recall:  0.840843720038351
f1:  0.7965485921889192


array([[0.72434018, 0.27565982],
       [0.15915628, 0.84084372]])

In [143]:
n_estimators = [5,20,50,100, 150, 200, 250, 300] # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10, 12, 14] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4, 5, 6] # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points

random_grid = {'n_estimators': n_estimators,
'max_features': max_features,
'max_depth': max_depth,
'min_samples_split': min_samples_split,
'min_samples_leaf': min_samples_leaf,
'bootstrap': bootstrap}

In [144]:
rf_random = RandomizedSearchCV(estimator = ran_for,param_distributions = random_grid,n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)

In [145]:
rf_random.fit(xtrain, ytrain)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      120],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 3, 4, 5, 6],
                                        'min_samples_split': [2, 6, 10, 12, 14],
                                        'n_estimators': [5, 20, 50, 100, 150,
                                                         200, 250, 300]},
                   random_state=35, verbose=2)

In [147]:
rf_random.best_params_

{'n_estimators': 300,
 'min_samples_split': 14,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 70,
 'bootstrap': True}

In [148]:
ran_for_tuned = RandomForestClassifier(n_estimators = 300, min_samples_split=14, min_samples_leaf=4, 
                                       max_features='sqrt', max_depth=70, bootstrap=True)
ran_for_tuned.fit(xtrain_smot, ytrain_smot)

ran_for_tuned_original_data_pred = ran_for.predict(xtest_smot)

ran_for_tuned_original_data_accuracy = accuracy_score(ytest_smot, ran_for_tuned_original_data_pred)
print('accuracy: ', ran_for_tuned_original_data_accuracy)

ran_for_tuned_original_data_precision = precision_score(ytest_smot, ran_for_tuned_original_data_pred)
print('precision: ', ran_for_tuned_original_data_precision)

ran_for_tuned_original_data_recall = recall_score(ytest_smot, ran_for_tuned_original_data_pred)
print('recall: ', ran_for_tuned_smot_recall)

ran_for_tuned_original_data_f1 = f1_score(ytest_smot, ran_for_tuned_original_data_pred)
print('f1: ', ran_for_tuned_original_data_f1)


accuracy:  0.920135527589545
precision:  0.9565217391304348
recall:  0.8791423001949318
f1:  0.9162011173184358


#### XGBoost

In [44]:
xg_boost = XGBRegressor()
xg_boost.fit(xtrain, ytrain)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [45]:
xgb_original_data_pred = xg_boost.predict(xtest)
xgb_original_data_pred = np.round(xgb_original_data_pred)

In [48]:
xgb_original_data_accuracy = accuracy_score(ytest, xgb_original_data_pred)
xgb_original_data_accuracy

0.7821387940841866

In [55]:
xgb_original_data_confusion_matrix = confusion_matrix(ytest, xgb_original_data_pred)
xgb_original_data_confusion_matrix.astype('float')/ xgb_original_data_confusion_matrix.sum(axis=1)[:,np.newaxis]

array([[0.87685113, 0.12314887],
       [0.47368421, 0.52631579]])

In [56]:
xgb_original_data_precision = precision_score(ytest, xgb_original_data_pred)
print(xgb_original_data_precision)

0.6127450980392157


In [57]:
xgb_original_data_recall = recall_score(ytest, xgb_original_data_pred)
print(xgb_original_data_recall)

0.5263157894736842


In [58]:
xgb_original_data_f1 = f1_score(ytest, xgb_original_data_pred)
print(xgb_original_data_f1)

0.5662514156285392


### Brief comparision (original data)

    model                Accuracy       Precision       Recall       F1 score

    Naive bayes          0.7224         0.507           0.802        0.621

    Logistic regression  0.816          0.259           0.194        0.221

    Random forest        0.795          0.25            0.172        0.203

    XGBoost              0.7633         0.611           0.462        0.526

Accuracy: quality of prediction

Prediction: quality of positive predictions

recall: ratio of actually positive values to precicted positive values

F1: harmonic mean of precision and recall 

---

# SMOTE

In [59]:
smot = SMOTE(random_state=101)
x_resamples, y_resamples = smot.fit_resample(original_data_copy[colu[1:len(colu)-1]], original_data_copy['Churn'])

In [60]:
x_resamples

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,29.850000,29.850000
1,1,0,1,0,34,1,1,0,1,1,1,0,0,0,1,1,1,56.950000,1889.500000
2,1,0,1,0,2,1,1,0,1,0,0,0,0,0,0,0,1,53.850000,108.150000
3,1,0,1,0,45,0,0,0,1,1,1,1,0,0,1,1,2,42.300000,1840.750000
4,0,0,1,0,2,1,1,1,0,1,0,0,0,0,0,0,0,70.700000,151.650000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10321,0,0,1,0,16,1,1,0,0,1,1,0,0,0,0,0,0,58.209558,937.864485
10322,1,0,0,0,54,1,1,1,1,0,0,0,0,1,1,0,2,109.528404,6109.574554
10323,0,0,1,0,1,1,1,1,0,1,0,0,0,0,0,0,0,70.114711,70.114711
10324,1,0,0,0,7,1,1,1,0,0,0,0,1,1,0,0,0,97.604865,666.618046


In [61]:
y_resamples

0        0
1        0
2        1
3        0
4        1
        ..
10321    1
10322    1
10323    1
10324    1
10325    1
Name: Churn, Length: 10326, dtype: int64

In [62]:
print(len(x_resamples),'\n----------\n')
for x in graph_colu:
    print(x)
    print(x_resamples[x].value_counts(),'\n')

10326 
----------

gender
0    5852
1    4474
Name: gender, dtype: int64 

SeniorCitizen
0    8923
1    1403
Name: SeniorCitizen, dtype: int64 

Partner
0    5195
1    5131
Name: Partner, dtype: int64 

Dependents
0    8106
1    2220
Name: Dependents, dtype: int64 

tenure
1     1270
2      469
3      385
72     362
4      322
      ... 
38      75
59      73
39      73
44      72
36      70
Name: tenure, Length: 72, dtype: int64 

PhoneService
1    9253
0    1073
Name: PhoneService, dtype: int64 

MultipleLines
1    5334
2    3952
0    1040
Name: MultipleLines, dtype: int64 

InternetService
1    5294
0    3350
2    1682
Name: InternetService, dtype: int64 

OnlineSecurity
0    6485
1    2159
2    1682
Name: OnlineSecurity, dtype: int64 

OnlineBackup
1    4827
0    3817
2    1682
Name: OnlineBackup, dtype: int64 

DeviceProtection
0    5751
1    2893
2    1682
Name: DeviceProtection, dtype: int64 

TechSupport
0    6456
1    2188
2    1682
Name: TechSupport, dtype: int64 

StreamingT

#### SMOTE-NC

In [63]:
xtrain.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges'],
      dtype='object')

In [64]:
smotnc = SMOTENC([0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17], random_state=101)
x_resamples_nc, y_resamples_nc = smotnc.fit_resample(original_data_copy[colu[1:len(colu)-1]], original_data_copy['Churn'])

In [65]:
x_resamples_nc

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,29.85,29.850000
1,1,0,1,0,34,1,1,0,1,1,1,0,0,0,1,1,1,56.95,1889.500000
2,1,0,1,0,2,1,1,0,1,0,0,0,0,0,0,0,1,53.85,108.150000
3,1,0,1,0,45,0,0,0,1,1,1,1,0,0,1,1,2,42.30,1840.750000
4,0,0,1,0,2,1,1,1,0,1,0,0,0,0,0,0,0,70.70,151.650000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10321,0,0,1,0,17,1,2,0,0,1,0,0,0,0,0,0,1,55.70,954.080850
10322,0,0,0,0,53,1,1,1,1,0,0,0,1,1,1,0,3,105.40,5773.064110
10323,0,0,1,0,1,1,1,1,0,1,0,0,0,0,0,0,0,70.10,70.114711
10324,1,0,1,0,6,1,1,1,0,0,0,0,1,1,0,0,0,96.25,593.414894


In [66]:
print(len(x_resamples_nc),'\n----------\n')
for x in graph_colu:
    print(x)
    print(x_resamples_nc[x].value_counts(),'\n')

10326 
----------

gender
1    5243
0    5083
Name: gender, dtype: int64 

SeniorCitizen
0    8479
1    1847
Name: SeniorCitizen, dtype: int64 

Partner
1    5853
0    4473
Name: Partner, dtype: int64 

Dependents
0    7829
1    2497
Name: Dependents, dtype: int64 

tenure
1     1334
2      479
3      370
72     364
4      317
      ... 
36      73
45      73
38      72
59      70
44      62
Name: tenure, Length: 72, dtype: int64 

PhoneService
1    9342
0     984
Name: PhoneService, dtype: int64 

MultipleLines
1    4883
2    4459
0     984
Name: MultipleLines, dtype: int64 

InternetService
1    5451
0    3144
2    1731
Name: InternetService, dtype: int64 

OnlineSecurity
0    6280
1    2315
2    1731
Name: OnlineSecurity, dtype: int64 

OnlineBackup
1    5330
0    3265
2    1731
Name: OnlineBackup, dtype: int64 

DeviceProtection
0    5274
1    3321
2    1731
Name: DeviceProtection, dtype: int64 

TechSupport
0    6207
1    2388
2    1731
Name: TechSupport, dtype: int64 

StreamingT

---

## Using SMOT data

In [67]:
xtrain_smot, xtest_smot, ytrain_smot, ytest_smot = train_test_split(x_resamples, y_resamples, test_size=0.20)

#### naive bayes smot

In [68]:
naive = GaussianNB()
model = naive.fit(xtrain_smot, ytrain_smot)
#naive_bayes_original_data_pred = naive.predict(xtest.iloc[0].to_numpy().reshape(1,-1))
naive_bayes_smot_pred = naive.predict(xtest_smot)

naive_bayes_smot_accuracy = accuracy_score(ytest_smot,naive_bayes_smot_pred)
print('accuracy: ', naive_bayes_smot_accuracy)

naive_bayes_smot_precision = precision_score(ytest_smot, naive_bayes_smot_pred)
print('precision: ', naive_bayes_smot_precision)

naive_bayes_smot_recall = recall_score(ytest_smot, naive_bayes_smot_pred)
print('recall: ', naive_bayes_smot_recall)

naive_bayes_smot_f1 = f1_score(ytest_smot, naive_bayes_smot_pred)
print('f1: ', naive_bayes_smot_f1)

naive_bayes_smot_confusion_matrix = confusion_matrix(ytest_smot, naive_bayes_smot_pred)
naive_bayes_smot_confusion_matrix.astype('float')/ naive_bayes_smot_confusion_matrix.sum(axis=1)[:, np.newaxis]

accuracy:  0.7894482090997096
precision:  0.7432098765432099
recall:  0.8801169590643275
f1:  0.8058902275769746


array([[0.7       , 0.3       ],
       [0.11988304, 0.88011696]])

#### Logistic regression smot

In [69]:
log = LogisticRegression(max_iter=1000000)
log.fit(xtrain_smot, ytrain_smot)
log_reg_smot_pred = log.predict(xtest_smot)

log_reg_smot_accuracy = accuracy_score(ytest_smot, log_reg_smot_pred)
print('accuracy: ', log_reg_smot_accuracy)

log_reg_smot_precision = precision_score(ytest_smot, log_reg_smot_pred)
print('precision: ', log_reg_smot_precision)

log_reg_smot_recall = recall_score(ytest_smot, log_reg_smot_pred)
print('recall: ', log_reg_smot_recall)

log_reg_smot_f1 = f1_score(ytest_smot, log_reg_smot_pred)
print('f1: ', log_reg_smot_f1)

log_reg_smot_confusion_matrix = confusion_matrix(ytest_smot, log_reg_smot_pred)
log_reg_smot_confusion_matrix.astype('float')/ log_reg_smot_confusion_matrix.sum(axis=1)[:, np.newaxis]

accuracy:  0.8112294288480155
precision:  0.797752808988764
recall:  0.8304093567251462
f1:  0.8137535816618909


array([[0.79230769, 0.20769231],
       [0.16959064, 0.83040936]])

#### Random forest smot

In [70]:
ran_for = RandomForestClassifier()
ran_for.fit(xtrain_smot, ytrain_smot)

ran_for_smot_pred = ran_for.predict(xtest_smot)

ran_for_smot_accuracy = accuracy_score(ytest_smot, ran_for_smot_pred)
print('accuracy: ', ran_for_smot_accuracy)

ran_for_smot_precision = precision_score(ytest_smot, ran_for_smot_pred)
print('precision: ', ran_for_smot_precision)

ran_for_smot_recall = recall_score(ytest_smot, ran_for_smot_pred)
print('recall: ', ran_for_smot_recall)

ran_for_smot_f1 = f1_score(ytest_smot, ran_for_smot_pred)
print('f1: ', ran_for_smot_f1)


ran_for_smot_confusion_matrix = confusion_matrix(ytest_smot, ran_for_smot_pred)
ran_for_smot_confusion_matrix.astype('float')/ ran_for_smot_confusion_matrix.sum(axis=1)[:, np.newaxis]

accuracy:  0.8441432720232332
precision:  0.8333333333333334
recall:  0.8576998050682261
f1:  0.845341018251681


array([[0.83076923, 0.16923077],
       [0.14230019, 0.85769981]])

### Hyperparameter tuning for Random forest

In [110]:
n_estimators = [5,20,50,100, 150, 200, 250, 300] # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10, 12, 14] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4, 5, 6] # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points

random_grid = {'n_estimators': n_estimators,
'max_features': max_features,
'max_depth': max_depth,
'min_samples_split': min_samples_split,
'min_samples_leaf': min_samples_leaf,
'bootstrap': bootstrap}

In [111]:
rf_random = RandomizedSearchCV(estimator = ran_for,param_distributions = random_grid,n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)

In [112]:
rf_random.fit(xtrain_smot, ytrain_smot)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      120],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 3, 4, 5, 6],
                                        'min_samples_split': [2, 6, 10, 12, 14],
                                        'n_estimators': [5, 20, 50, 100, 150,
                                                         200, 250, 300]},
                   random_state=35, verbose=2)

In [113]:
rf_random.best_params_

{'n_estimators': 250,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 120,
 'bootstrap': True}

In [114]:
ran_for_tuned = RandomForestClassifier(n_estimators = 250, min_samples_split=2, min_samples_leaf=1, 
                                       max_features='sqrt', max_depth=120, bootstrap=True)
ran_for_tuned.fit(xtrain_smot, ytrain_smot)

ran_for_tuned_smot_pred = ran_for.predict(xtest_smot)

ran_for_tuned_smot_accuracy = accuracy_score(ytest_smot, ran_for_smot_pred)
print('accuracy: ', ran_for_tuned_smot_accuracy)

ran_for_tuned_smot_precision = precision_score(ytest_smot, ran_for_tuned_smot_pred)
print('precision: ', ran_for_tuned_smot_precision)

ran_for_tuned_smot_recall = recall_score(ytest_smot, ran_for_tuned_smot_pred)
print('recall: ', ran_for_tuned_smot_recall)

ran_for_tuned_smot_f1 = f1_score(ytest_smot, ran_for_tuned_smot_pred)
print('f1: ', ran_for_tuned_smot_f1)


accuracy:  0.8441432720232332
precision:  0.9565217391304348
recall:  0.8791423001949318
f1:  0.9162011173184358


#### xgboost smot

In [71]:
xg_boost = XGBRegressor()
xg_boost.fit(xtrain_smot, ytrain_smot)

xgb_smot_prediction = xg_boost.predict(xtest_smot)
xgb_smot_pred = np.round(xgb_smot_prediction)

xgb_smot_accuracy = accuracy_score(ytest_smot, xgb_smot_pred)
print('accuracy: ', xgb_smot_accuracy)

xgb_smot_precision = precision_score(ytest_smot, xgb_smot_pred)
print('precision: ', xgb_smot_precision)

xgb_smot_recall = recall_score(ytest_smot, xgb_smot_pred)
print('recall: ', xgb_smot_recall)

xgb_smot_f1 = f1_score(ytest_smot, xgb_smot_pred)
print('f1: ', xgb_smot_f1)

xgb_smot_confusion_matrix = confusion_matrix(ytest_smot, xgb_smot_pred)
xgb_smot_confusion_matrix.astype('float')/ xgb_smot_confusion_matrix.sum(axis=1)[:,np.newaxis]

accuracy:  0.8397870280735721
precision:  0.8377065111758989
recall:  0.8401559454191033
f1:  0.8389294403892944


array([[0.83942308, 0.16057692],
       [0.15984405, 0.84015595]])

### Brief comparision (original data)

    model                Accuracy       Precision       Recall       F1 score

    Naive bayes          0.7224         0.507           0.802        0.621

    Logistic regression  0.816          0.259           0.194        0.221

    Random forest        0.795          0.25            0.172        0.203

    XGBoost              0.7633         0.611           0.462        0.526

### Brief comparision (SMOT data)

    model                Accuracy       Precision       Recall       F1 score

    Naive bayes          0.733          0.732           0.854        0.788       

    Logistic regression  0.862          0.259           0.863        0.831

    Random forest        0.849          0.25            0.847        0.847

    XGBoost              0.846          0.838           0.853        0.845

---

## Using SMOTE-NC data

In [72]:
xtrain_smotnc, xtest_smotnc, ytrain_smotnc, ytest_smotnc = train_test_split(x_resamples_nc, y_resamples_nc, test_size=0.20)

#### naive bayes smotnc

In [73]:
naive = GaussianNB()
model = naive.fit(xtrain_smotnc, ytrain_smotnc)
#naive_bayes_original_data_pred = naive.predict(xtest.iloc[0].to_numpy().reshape(1,-1))
naive_bayes_smotnc_pred = naive.predict(xtest_smotnc)

naive_bayes_smotnc_accuracy = accuracy_score(ytest_smotnc,naive_bayes_smotnc_pred)
print('accuracy: ', naive_bayes_smotnc_accuracy)

naive_bayes_smotnc_precision = precision_score(ytest_smotnc, naive_bayes_smotnc_pred)
print('precision: ', naive_bayes_smot_precision)

naive_bayes_smotnc_recall = recall_score(ytest_smotnc, naive_bayes_smotnc_pred)
print('recall: ', naive_bayes_smotnc_recall)

naive_bayes_smotnc_f1 = f1_score(ytest_smotnc, naive_bayes_smotnc_pred)
print('f1: ', naive_bayes_smotnc_f1)

naive_bayes_smotnc_confusion_matrix = confusion_matrix(ytest_smotnc, naive_bayes_smotnc_pred)
naive_bayes_smotnc_confusion_matrix.astype('float')/ naive_bayes_smotnc_confusion_matrix.sum(axis=1)[:, np.newaxis]

accuracy:  0.7429816069699903
precision:  0.7432098765432099
recall:  0.8475551294343241
f1:  0.7690300130491518


array([[0.63636364, 0.36363636],
       [0.15244487, 0.84755513]])

#### Logistic regression smotnc

In [74]:
log = LogisticRegression(max_iter=1000000)
log.fit(xtrain_smotnc, ytrain_smotnc)
log_reg_smotnc_pred = log.predict(xtest_smotnc)

log_reg_smotnc_accuracy = accuracy_score(ytest_smotnc, log_reg_smotnc_pred)
print('accuracy: ', log_reg_smotnc_accuracy)

log_reg_smotnc_precision = precision_score(ytest_smotnc, log_reg_smotnc_pred)
print('precision: ', log_reg_smot_precision)

log_reg_smotnc_recall = recall_score(ytest_smotnc, log_reg_smotnc_pred)
print('recall: ', log_reg_smotnc_recall)

log_reg_smotnc_f1 = f1_score(ytest_smotnc, log_reg_smotnc_pred)
print('f1: ', log_reg_smotnc_f1)

log_reg_smotnc_confusion_matrix = confusion_matrix(ytest_smotnc, log_reg_smotnc_pred)
log_reg_smotnc_confusion_matrix.astype('float')/ log_reg_smotnc_confusion_matrix.sum(axis=1)[:, np.newaxis]

accuracy:  0.7831558567279767
precision:  0.797752808988764
recall:  0.840843720038351
f1:  0.7965485921889192


array([[0.72434018, 0.27565982],
       [0.15915628, 0.84084372]])

#### Random forest smotnc

In [75]:
ran_for = RandomForestClassifier()
ran_for.fit(xtrain_smotnc, ytrain_smotnc)

ran_for_smotnc_pred = ran_for.predict(xtest_smotnc)

ran_for_smotnc_accuracy = accuracy_score(ytest_smotnc, ran_for_smotnc_pred)
print('accuracy: ', ran_for_smotnc_accuracy)

ran_for_smotnc_precision = precision_score(ytest_smotnc, ran_for_smot_pred)
print('precision: ', ran_for_smotnc_precision)

ran_for_smotnc_recall = recall_score(ytest_smotnc, ran_for_smot_pred)
print('recall: ', ran_for_smotnc_recall)

ran_for_smotnc_f1 = f1_score(ytest_smotnc, ran_for_smot_pred)
print('f1: ', ran_for_smotnc_f1)

ran_for_smotnc_confusion_matrix = confusion_matrix(ytest_smotnc, ran_for_smotnc_pred)
ran_for_smotnc_confusion_matrix.astype('float')/ ran_for_smotnc_confusion_matrix.sum(axis=1)[:, np.newaxis]

accuracy:  0.8267182962245886
precision:  0.4990530303030303
recall:  0.5052732502396932
f1:  0.5021438780371605


array([[0.7888563 , 0.2111437 ],
       [0.13614573, 0.86385427]])

#### xgboost smot

In [76]:
xg_boost = XGBRegressor()
xg_boost.fit(xtrain_smotnc, ytrain_smotnc)

xgb_smotnc_prediction = xg_boost.predict(xtest_smotnc)
xgb_smotnc_prediction = np.round(xgb_smotnc_prediction)

xgb_smotnc_accuracy = accuracy_score(ytest_smotnc, xgb_smotnc_prediction)
print('accuracy: ', xgb_smotnc_accuracy)

xgb_smotnc_precision = precision_score(ytest_smotnc, xgb_smot_pred)
print('precision: ', xgb_smotnc_precision)

xgb_smotnc_recall = recall_score(ytest_smotnc, xgb_smot_pred)
print('recall: ', xgb_smotnc_recall)

xgb_smotnc_f1 = f1_score(ytest_smotnc, xgb_smot_pred)
print('f1: ', xgb_smotnc_f1)

xgb_smotnc_confusion_matrix = confusion_matrix(ytest_smotnc, xgb_smotnc_prediction)
xgb_smotnc_confusion_matrix.astype('float')/ xgb_smotnc_confusion_matrix.sum(axis=1)[:,np.newaxis]

accuracy:  0.8199419167473379
precision:  0.4868804664723032
recall:  0.48034515819750717
f1:  0.4835907335907336


array([[0.77321603, 0.22678397],
       [0.13422819, 0.86577181]])

---

## performance matrixes

In [98]:
accuracy_matrix

,Model,original_data,SMOT,SMOTNC
0,Naive bayes,0.715586,0.789448,0.742982
1,Logistic Regression,0.812856,0.811229,0.783156
2,Random Forest,0.788965,0.844143,0.826718
3,XGBoost,0.782139,0.839787,0.819942


In [99]:
precision_matrix

,Model,original_data,SMOT,SMOTNC
0,Naive bayes,0.484197,0.74321,0.703822
1,Logistic Regression,0.685279,0.797753,0.756687
2,Random Forest,0.645251,0.833333,0.499053
3,XGBoost,0.612745,0.837707,0.48688


In [100]:
recall_matrix

,Model,original_data,SMOT,SMOTNC
0,Naive bayes,0.806316,0.880117,0.847555
1,Logistic Regression,0.568421,0.830409,0.840844
2,Random Forest,0.486316,0.8577,0.505273
3,XGBoost,0.526316,0.840156,0.480345


In [101]:
f1_matrix

,Model,original_data,SMOT,SMOTNC
0,Naive bayes,0.605055,0.80589,0.76903
1,Logistic Regression,0.621404,0.813754,0.796549
2,Random Forest,0.554622,0.845341,0.502144
3,XGBoost,0.566251,0.838929,0.483591


---

# Ranking overall best performences

    accuracy: 
    (best)    Logistic regression  
    (runner)  Random forest        

    precision:
            XGBoost
            logistic regression

    recall:
            Logistic regression
            Naive bayes / XGBoost

    F1 score:
            Random forest / XGBoost
            Logistic regression

### Comparing the models trained on Raw, SMOTE and SMOTE-NC data, SMOTE performs best when paired with Logistic Regression

---

In [138]:
precision_matrix = pd.DataFrame(columns=[['Model','original_data', 'SMOT', 'SMOTNC']])
precision_matrix

,Model,original_data,SMOT,SMOTNC


In [139]:
precision_matrix.loc[0, 'Model'] = 'Naive bayes'
precision_matrix.loc[0, 'original_data'] = naive_bayes_original_data_precision
precision_matrix.loc[0, 'SMOT'] = naive_bayes_smot_precision
precision_matrix.loc[0, 'SMOTNC'] = naive_bayes_smotnc_precision

In [140]:
precision_matrix.loc[1, 'Model'] = 'Logistic Regression'
precision_matrix.loc[1, 'original_data'] = log_reg_original_data_precision
precision_matrix.loc[1, 'SMOT'] = log_reg_smot_precision
precision_matrix.loc[1, 'SMOTNC'] = log_reg_smotnc_precision

In [141]:
precision_matrix.loc[2, 'Model'] = 'Random Forest'
precision_matrix.loc[2, 'original_data'] = ran_for_original_data_precision
precision_matrix.loc[2, 'SMOT'] = ran_for_smot_precision
precision_matrix.loc[2, 'SMOTNC'] = ran_for_smotnc_precision

In [150]:
precision_matrix.loc[3, 'Model'] = 'Random Forest Tuned'
precision_matrix.loc[3, 'original_data'] = ran_for_tuned_original_data_precision
precision_matrix.loc[3, 'SMOT'] = ran_for_tuned_smot_precision
precision_matrix.loc[3, 'SMOTNC'] = ran_for_tuned_smotnc_precision

NameError: name 'ran_for_tuned_smotnc_precision' is not defined

#### Random forest smotnc

In [75]:
ran_for = RandomForestClassifier()
ran_for.fit(xtrain_smotnc, ytrain_smotnc)

ran_for_smotnc_pred = ran_for.predict(xtest_smotnc)

ran_for_smotnc_accuracy = accuracy_score(ytest_smotnc, ran_for_smotnc_pred)
print('accuracy: ', ran_for_smotnc_accuracy)

ran_for_smotnc_precision = precision_score(ytest_smotnc, ran_for_smot_pred)
print('precision: ', ran_for_smotnc_precision)

ran_for_smotnc_recall = recall_score(ytest_smotnc, ran_for_smot_pred)
print('recall: ', ran_for_smotnc_recall)

ran_for_smotnc_f1 = f1_score(ytest_smotnc, ran_for_smot_pred)
print('f1: ', ran_for_smotnc_f1)

ran_for_smotnc_confusion_matrix = confusion_matrix(ytest_smotnc, ran_for_smotnc_pred)
ran_for_smotnc_confusion_matrix.astype('float')/ ran_for_smotnc_confusion_matrix.sum(axis=1)[:, np.newaxis]

accuracy:  0.8267182962245886
precision:  0.4990530303030303
recall:  0.5052732502396932
f1:  0.5021438780371605


array([[0.7888563 , 0.2111437 ],
       [0.13614573, 0.86385427]])

#### xgboost smot

In [76]:
xg_boost = XGBRegressor()
xg_boost.fit(xtrain_smotnc, ytrain_smotnc)

xgb_smotnc_prediction = xg_boost.predict(xtest_smotnc)
xgb_smotnc_prediction = np.round(xgb_smotnc_prediction)

xgb_smotnc_accuracy = accuracy_score(ytest_smotnc, xgb_smotnc_prediction)
print('accuracy: ', xgb_smotnc_accuracy)

xgb_smotnc_precision = precision_score(ytest_smotnc, xgb_smot_pred)
print('precision: ', xgb_smotnc_precision)

xgb_smotnc_recall = recall_score(ytest_smotnc, xgb_smot_pred)
print('recall: ', xgb_smotnc_recall)

xgb_smotnc_f1 = f1_score(ytest_smotnc, xgb_smot_pred)
print('f1: ', xgb_smotnc_f1)

xgb_smotnc_confusion_matrix = confusion_matrix(ytest_smotnc, xgb_smotnc_prediction)
xgb_smotnc_confusion_matrix.astype('float')/ xgb_smotnc_confusion_matrix.sum(axis=1)[:,np.newaxis]

accuracy:  0.8199419167473379
precision:  0.4868804664723032
recall:  0.48034515819750717
f1:  0.4835907335907336


array([[0.77321603, 0.22678397],
       [0.13422819, 0.86577181]])

---

## performance matrixes

In [98]:
accuracy_matrix

,Model,original_data,SMOT,SMOTNC
0,Naive bayes,0.715586,0.789448,0.742982
1,Logistic Regression,0.812856,0.811229,0.783156
2,Random Forest,0.788965,0.844143,0.826718
3,XGBoost,0.782139,0.839787,0.819942


In [99]:
precision_matrix

,Model,original_data,SMOT,SMOTNC
0,Naive bayes,0.484197,0.74321,0.703822
1,Logistic Regression,0.685279,0.797753,0.756687
2,Random Forest,0.645251,0.833333,0.499053
3,XGBoost,0.612745,0.837707,0.48688


In [100]:
recall_matrix

,Model,original_data,SMOT,SMOTNC
0,Naive bayes,0.806316,0.880117,0.847555
1,Logistic Regression,0.568421,0.830409,0.840844
2,Random Forest,0.486316,0.8577,0.505273
3,XGBoost,0.526316,0.840156,0.480345


In [101]:
f1_matrix

,Model,original_data,SMOT,SMOTNC
0,Naive bayes,0.605055,0.80589,0.76903
1,Logistic Regression,0.621404,0.813754,0.796549
2,Random Forest,0.554622,0.845341,0.502144
3,XGBoost,0.566251,0.838929,0.483591


---

# Ranking overall best performences

    accuracy: 
    (best)    Logistic regression  
    (runner)  Random forest        

    precision:
            XGBoost
            logistic regression

    recall:
            Logistic regression
            Naive bayes / XGBoost

    F1 score:
            Random forest / XGBoost
            Logistic regression

### Comparing the models trained on Raw, SMOTE and SMOTE-NC data, SMOTE performs best when paired with Logistic Regression

---

In [138]:
precision_matrix = pd.DataFrame(columns=[['Model','original_data', 'SMOT', 'SMOTNC']])
precision_matrix

,Model,original_data,SMOT,SMOTNC


In [139]:
precision_matrix.loc[0, 'Model'] = 'Naive bayes'
precision_matrix.loc[0, 'original_data'] = naive_bayes_original_data_precision
precision_matrix.loc[0, 'SMOT'] = naive_bayes_smot_precision
precision_matrix.loc[0, 'SMOTNC'] = naive_bayes_smotnc_precision

In [140]:
precision_matrix.loc[1, 'Model'] = 'Logistic Regression'
precision_matrix.loc[1, 'original_data'] = log_reg_original_data_precision
precision_matrix.loc[1, 'SMOT'] = log_reg_smot_precision
precision_matrix.loc[1, 'SMOTNC'] = log_reg_smotnc_precision

In [141]:
precision_matrix.loc[2, 'Model'] = 'Random Forest'
precision_matrix.loc[2, 'original_data'] = ran_for_original_data_precision
precision_matrix.loc[2, 'SMOT'] = ran_for_smot_precision
precision_matrix.loc[2, 'SMOTNC'] = ran_for_smotnc_precision

In [150]:
precision_matrix.loc[3, 'Model'] = 'Random Forest Tuned'
precision_matrix.loc[3, 'original_data'] = ran_for_tuned_original_data_precision
precision_matrix.loc[3, 'SMOT'] = ran_for_tuned_smot_precision
precision_matrix.loc[3, 'SMOTNC'] = ran_for_tuned_smotnc_precision

NameError: name 'ran_for_tuned_smotnc_precision' is not defined

In [ ]:
precision_matrix.loc[4, 'Model'] = 'XGBoost'
precision_matrix.loc[4, 'original_data'] = xgb_original_data_precision
precision_matrix.loc[4, 'SMOT'] = xgb_smot_precision
precision_matrix.loc[4 'SMOTNC'] = xgb_smotnc_precision

In [ ]:
accuracy_matrix = pd.DataFrame(columns=[['Model','original_data', 'SMOT', 'SMOTNC']])
accuracy_matrix

In [ ]:
accuracy_matrix.loc[0, 'Model'] = 'Naive bayes'
accuracy_matrix.loc[0, 'original_data'] = naive_bayes_original_data_accuracy
accuracy_matrix.loc[0, 'SMOT'] = naive_bayes_smot_accuracy
accuracy_matrix.loc[0, 'SMOTNC'] = naive_bayes_smotnc_accuracy

In [ ]:
accuracy_matrix.loc[1, 'Model'] = 'Logistic Regression'
accuracy_matrix.loc[1, 'original_data'] = log_reg_original_data_accuracy
accuracy_matrix.loc[1, 'SMOT'] = log_reg_smot_accuracy
accuracy_matrix.loc[1, 'SMOTNC'] = log_reg_smotnc_accuracy

In [ ]:
accuracy_matrix.loc[2, 'Model'] = 'Random Forest'
accuracy_matrix.loc[2, 'original_data'] = ran_for_original_data_accuracy
accuracy_matrix.loc[2, 'SMOT'] = ran_for_smot_accuracy
accuracy_matrix.loc[2, 'SMOTNC'] = ran_for_smotnc_accuracy

In [ ]:
accuracy_matrix.loc[3, 'Model'] = 'Random Forest'
accuracy_matrix.loc[3, 'original_data'] = ran_for_tuned_original_data_accuracy
accuracy_matrix.loc[3, 'SMOT'] = ran_for_tuned_smot_accuracy
accuracy_matrix.loc[3, 'SMOTNC'] = ran_for_tuned_smotnc_accuracy

In [ ]:
accuracy_matrix.loc[4, 'Model'] = 'XGBoost'
accuracy_matrix.loc[4, 'original_data'] = xgb_original_data_accuracy
accuracy_matrix.loc[4, 'SMOT'] = xgb_smot_accuracy
accuracy_matrix.loc[4, 'SMOTNC'] = xgb_smotnc_accuracy

In [ ]:
recall_matrix = pd.DataFrame(columns=[['Model','original_data', 'SMOT', 'SMOTNC']])
recall_matrix

In [ ]:
recall_matrix.loc[0, 'Model'] = 'Naive bayes'
recall_matrix.loc[0, 'original_data'] = naive_bayes_original_data_recall
recall_matrix.loc[0, 'SMOT'] = naive_bayes_smot_recall
recall_matrix.loc[0, 'SMOTNC'] = naive_bayes_smotnc_recall

In [ ]:
recall_matrix.loc[1, 'Model'] = 'Logistic Regression'
recall_matrix.loc[1, 'original_data'] = log_reg_original_data_recall
recall_matrix.loc[1, 'SMOT'] = log_reg_smot_recall
recall_matrix.loc[1, 'SMOTNC'] = log_reg_smotnc_recall

In [ ]:
recall_matrix.loc[2, 'Model'] = 'Random Forest'
recall_matrix.loc[2, 'original_data'] = ran_for_original_data_recall
recall_matrix.loc[2, 'SMOT'] = ran_for_smot_recall
recall_matrix.loc[2, 'SMOTNC'] = ran_for_smotnc_recall

In [ ]:
recall_matrix.loc[3, 'Model'] = 'Random Forest'
recall_matrix.loc[3, 'original_data'] = ran_for_tuned_original_data_recall
recall_matrix.loc[3, 'SMOT'] = ran_for_tuned_smot_recall
recall_matrix.loc[3, 'SMOTNC'] = ran_for_tuned_smotnc_recall

In [ ]:
recall_matrix.loc[4, 'Model'] = 'XGBoost'
recall_matrix.loc[4, 'original_data'] = xgb_original_data_recall
recall_matrix.loc[4, 'SMOT'] = xgb_smot_recall
recall_matrix.loc[4, 'SMOTNC'] = xgb_smotnc_recall

In [ ]:
f1_matrix = pd.DataFrame(columns=[['Model','original_data', 'SMOT', 'SMOTNC']])
f1_matrix

In [ ]:
f1_matrix.loc[0, 'Model'] = 'Naive bayes'
f1_matrix.loc[0, 'original_data'] = naive_bayes_original_data_f1
f1_matrix.loc[0, 'SMOT'] = naive_bayes_smot_f1
f1_matrix.loc[0, 'SMOTNC'] = naive_bayes_smotnc_f1

In [ ]:
f1_matrix.loc[1, 'Model'] = 'Logistic Regression'
f1_matrix.loc[1, 'original_data'] = log_reg_original_data_f1
f1_matrix.loc[1, 'SMOT'] = log_reg_smot_f1
f1_matrix.loc[1, 'SMOTNC'] = log_reg_smotnc_f1

In [ ]:
f1_matrix.loc[2, 'Model'] = 'Random Forest'
f1_matrix.loc[2, 'original_data'] = ran_for_original_data_f1
f1_matrix.loc[2, 'SMOT'] = ran_for_smot_f1
f1_matrix.loc[2, 'SMOTNC'] = ran_for_smotnc_f1

In [ ]:
f1_matrix.loc[3, 'Model'] = 'Random Forest'
f1_matrix.loc[3, 'original_data'] = ran_for_tuned_original_data_f1
f1_matrix.loc[3, 'SMOT'] = ran_for_tuned_smot_f1
f1_matrix.loc[3, 'SMOTNC'] = ran_for_tuned_smotnc_f1

In [ ]:
f1_matrix.loc[4, 'Model'] = 'XGBoost'
f1_matrix.loc[4, 'original_data'] = xgb_original_data_f1
f1_matrix.loc[4, 'SMOT'] = xgb_smot_f1
f1_matrix.loc[4, 'SMOTNC'] = xgb_smotnc_f1